In [1]:
# 导入必需的库
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torch.optim import SGD,Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import samna
from tonic.datasets import NMNIST
from tonic.transforms import ToFrame
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetwork
from collections import Counter

In [2]:
# 数据准备
root_dir = "./NMNIST"
try:
    from tonic.datasets.nmnist import NMNIST
except ImportError:
    !pip install tonic
    from tonic.datasets.nmnist import NMNIST

# 下载 N-MNIST 数据集
_ = NMNIST(save_to=root_dir, train=True)
# _ = NMNIST(save_to=root_dir, train=False)

# 使用 Tonic 转换事件数据为帧
to_frame = ToFrame(sensor_size=NMNIST.sensor_size, n_time_bins=1)
cnn_train_dataset = NMNIST(save_to=root_dir, train=True, transform=to_frame)
cnn_test_dataset = NMNIST(save_to=root_dir, train=False, transform=to_frame)

# 打印一下数据的形状
sample_data, label = cnn_train_dataset[0]
print(f"The transformed array is in shape [Time-Step, Channel, Height, Width] --> {sample_data.shape}")

The transformed array is in shape [Time-Step, Channel, Height, Width] --> (1, 2, 34, 34)


In [3]:
# 定义 CNN 模型
cnn = nn.Sequential(
    nn.Conv2d(in_channels=2, out_channels=8, kernel_size=(3, 3), padding=(1, 1), bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=(1, 1), bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=(1, 1), stride=(2, 2), bias=False),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(16 * 4 * 4, 10, bias=False),
    nn.ReLU(),
)

# 初始化模型权重
for layer in cnn.modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_normal_(layer.weight.data)

# 训练和验证 CNN
epochs = 1
lr = 1e-3
batch_size = 64
num_workers = 16
device = "cuda" if torch.cuda.is_available() else "cpu"
shuffle = True

cnn = cnn.to(device=device)

cnn_train_dataloader = DataLoader(cnn_train_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=shuffle)
cnn_test_dataloader = DataLoader(cnn_test_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=shuffle)

optimizer = Adam(params=cnn.parameters(), lr=lr)
criterion = CrossEntropyLoss()

/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
device

'cpu'

In [13]:
for e in range(epochs):
    # Train the model
    train_p_bar = tqdm(cnn_train_dataloader)
    for data, label in train_p_bar:
        data = data.squeeze(dim=1).to(dtype=torch.float, device=device)
        label = label.to(dtype=torch.long, device=device)
        optimizer.zero_grad()
        output = cnn(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_p_bar.set_description(f"Epoch {e} - Training Loss: {round(loss.item(), 4)}")

    # Validate the model
    correct_predictions = []
    with torch.no_grad():
        test_p_bar = tqdm(cnn_test_dataloader)
        for data, label in test_p_bar:
            data = data.squeeze(dim=1).to(dtype=torch.float, device=device)
            label = label.to(dtype=torch.long, device=device)
            output = cnn(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct_predictions.append(pred.eq(label.view_as(pred)))
            test_p_bar.set_description(f"Epoch {e} - Testing Model...")

        correct_predictions = torch.cat(correct_predictions)
        print(f"Epoch {e} - accuracy: {correct_predictions.sum().item()/(len(correct_predictions))*100}%")

Epoch 0 - Testing Model...: 100%|██████████| 156/156 [00:25<00:00,  6.07it/s]

Epoch 0 - accuracy: 74.70953525641025%


In [5]:
import torchvision.models as models
reset18=models.resnet18(pretrained=True)
reset18=reset18.to(device)

/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/taco/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 16.4MB/s]


In [6]:
class resnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1=nn.Sequential(   nn.Conv2d(in_channels=2, out_channels=3, kernel_size=(3, 3), padding=(1, 1), bias=False),
                                 nn.ReLU(),
                                 nn.AvgPool2d(2, 2),)
        self.l2=models.resnet18(pretrained=True)
        self.l3=nn.Linear(1000,10);
    def forward(self,x):
        x=self.l1(x)
        x=self.l2(x)
        x=self.l3(x)
        return x

In [7]:
resnet18=resnet()

In [8]:
resnet18=resnet18.to(device)

In [9]:

# 训练和验证 CNN
epochs = 1
lr = 1e-3
batch_size = 32
num_workers = 22
device = "cuda" if torch.cuda.is_available() else "cpu"
shuffle = True

#cnn = cnn.to(device=device)

cnn_train_dataloader = DataLoader(cnn_train_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=shuffle)
cnn_test_dataloader = DataLoader(cnn_test_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=shuffle)

optimizer = Adam(params=resnet18.parameters(), lr=lr)
criterion = CrossEntropyLoss()

/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 22 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
for e in range(epochs):
    # Train the model
    train_p_bar = tqdm(cnn_train_dataloader)
    for data, label in train_p_bar:
        data = data.squeeze(dim=1).to(dtype=torch.float, device=device)
        label = label.to(dtype=torch.long, device=device)
        optimizer.zero_grad()
        output = resnet18(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_p_bar.set_description(f"Epoch {e} - Training Loss: {round(loss.item(), 4)}")

    # Validate the model
    correct_predictions = []
    with torch.no_grad():
        test_p_bar = tqdm(cnn_test_dataloader)
        for data, label in test_p_bar:
            data = data.squeeze(dim=1).to(dtype=torch.float, device=device)
            label = label.to(dtype=torch.long, device=device)
            output = resnet18(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct_predictions.append(pred.eq(label.view_as(pred)))
            test_p_bar.set_description(f"Epoch {e} - Testing Model...")

        correct_predictions = torch.cat(correct_predictions)
        print(f"Epoch {e} - accuracy: {correct_predictions.sum().item()/(len(correct_predictions))*100}%")

  0%|          | 0/1875 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torch/__init__.py", line 1600, in <module>
    from . import masked
  File "/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torch/masked/__init__.py", line 1, in <module>
    from .maskedtensor.core import is_masked_tensor, MaskedTensor
  File "/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/torch/masked/maskedtensor/__init__.py", line 8, in <module>
    from .unary import _apply_native_unary, _is_native_unary
  File "/usr

KeyboardInterrupt: 

In [31]:
# CNN 转 SNN
snn_convert = from_model(model=cnn, input_shape=(2, 34, 34), batch_size=batch_size).spiking_model

# 转换后的 SNN 测试
n_time_steps = 100
to_raster = ToFrame(sensor_size=NMNIST.sensor_size, n_time_bins=n_time_steps)
snn_test_dataset = NMNIST(save_to=root_dir, train=False, transform=to_raster)
snn_test_dataloader = DataLoader(snn_test_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
snn_convert = snn_convert.to(device)

correct_predictions = []
with torch.no_grad():
    test_p_bar = tqdm(snn_test_dataloader)
    for data, label in test_p_bar:
        data = data.reshape(-1, 2, 34, 34).to(dtype=torch.float, device=device)
        label = label.to(dtype=torch.long, device=device)
        output = snn_convert(data)
        output = output.reshape(batch_size, n_time_steps, -1)
        output = output.sum(dim=1)
        pred = output.argmax(dim=1, keepdim=True)
        correct_predictions.append(pred.eq(label.view_as(pred)))
        test_p_bar.set_description(f"Testing SNN Model...")

    correct_predictions = torch.cat(correct_predictions)
    print(f"accuracy of converted SNN: {correct_predictions.sum().item()/(len(correct_predictions))*100}%")


Testing SNN Model...: 100%|██████████| 312/312 [00:41<00:00,  7.43it/s]

accuracy of converted SNN: 73.23717948717949%


In [ ]:
#It's similar to the cnn

In [11]:
# res18 转 SNN
snn_convert = from_model(model=resnet18, input_shape=(2, 34, 34), batch_size=batch_size).spiking_model

# 转换后的 SNN 测试
n_time_steps = 200
to_raster = ToFrame(sensor_size=NMNIST.sensor_size, n_time_bins=n_time_steps)
snn_test_dataset = NMNIST(save_to=root_dir, train=False, transform=to_raster)
snn_test_dataloader = DataLoader(snn_test_dataset, batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
snn_convert = snn_convert.to(device)

correct_predictions = []
with torch.no_grad():
    test_p_bar = tqdm(snn_test_dataloader)
    for data, label in test_p_bar:
        data = data.reshape(-1, 2, 34, 34).to(dtype=torch.float, device=device)
        label = label.to(dtype=torch.long, device=device)
        output = snn_convert(data)
        output = output.reshape(batch_size, n_time_steps, -1)
        output = output.sum(dim=1)
        pred = output.argmax(dim=1, keepdim=True)
        correct_predictions.append(pred.eq(label.view_as(pred)))
        test_p_bar.set_description(f"Testing SNN Model...")

    correct_predictions = torch.cat(correct_predictions)
    print(f"accuracy of converted SNN: {correct_predictions.sum().item()/(len(correct_predictions))*100}%")


Testing SNN Model...: 100%|██████████| 312/312 [23:09<00:00,  4.45s/it]

accuracy of converted SNN: 9.975961538461538%


In [ ]:
# 部署 SNN 到 Devkit
cpu_snn = snn_convert.to(device="cpu")
devices = samna.device.get_unopened_devices()
if len(devices) == 0:
    raise Exception("No devices found")
my_board = samna.device.open_device(devices[0])

# 使用设备信息部署到 DevKit
devkit_name = "speck2fdevkit"
dynapcnn = DynapcnnNetwork(snn=cpu_snn, input_shape=(2, 34, 34), discretize=True, dvs_input=False)
dynapcnn.to(device=devkit_name, chip_layers_ordering="auto")

print(f"The SNN is deployed on the core: {dynapcnn.chip_layers_ordering}")

AttributeError: 'samna.speck2fBoards.Speck2fDevKit' object has no attribute 'devkit_name'

In [3]:
devices = samna.device.get_unopened_devices()
devices

[]